In [1]:
import pandas as pd
import numpy as np

import os
import urllib
import json

import datetime
import csv
import time

from io import BytesIO

try:
    import Image
except ImportError:
    from PIL import Image
import pytesseract

%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
app_id = '125258161474215'
app_secret = '7340f93f8be2a23a8858e8ae24324947'

access_token = app_id + "|" + app_secret
print(access_token)

125258161474215|7340f93f8be2a23a8858e8ae24324947


In [3]:
posts = pd.read_csv('./2016-10-facebook-fact-check/facebook-fact-check.csv')
posts.shape

(2282, 12)

In [5]:
# ABC News Politics: mainstream
abc_posts_mastk = posts['Page'].str.contains('ABC')
abc_posts = posts[abc_posts_mastk]
print(abc_posts.shape)

(200, 12)


In [32]:
true_abc_posts_mask = (abc_posts['Rating'] == 'mostly true')
true_abc_posts = abc_posts[true_abc_posts_mask]
print(true_abc_posts.shape)
true_abc_posts.sort_values(by='share_count', ascending=False).head(5)

(172, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
187,184096565021911,1042051285893097,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-27,video,mostly true,yes,1104.0,1420.0,230.0
4,184096565021911,1035352946562931,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-19,video,mostly true,NaN,568.0,3188.0,2815.0
188,184096565021911,1042083579223201,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-27,video,mostly true,yes,375.0,915.0,395.0
51,184096565021911,1036496423115250,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-20,link,mostly true,NaN,339.0,430.0,231.0
22,184096565021911,1035656723199220,mainstream,ABC News Politics,https://www.facebook.com/ABCNewsPolitics/posts...,2016-09-19,video,mostly true,NaN,289.0,674.0,243.0


In [33]:
false_abc_posts_mask = (abc_posts['Rating'] == 'mostly false')
false_abc_posts = abc_posts[false_abc_posts_mask]
print(false_abc_posts.shape)
false_abc_posts.sort_values(by='share_count', ascending=False).head(5)

(0, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count


In [6]:
# Addicting Info: left
addicting_posts_mastk = posts['Page'].str.contains('Addicting')
addicting_posts = posts[addicting_posts_mastk]
print(addicting_posts.shape)

(140, 12)


In [30]:
true_addicting_posts_mask = (addicting_posts['Rating'] == 'mostly true')
true_addicting_posts = addicting_posts[true_addicting_posts_mask]
print(true_addicting_posts.shape)
true_addicting_posts.sort_values(by='share_count', ascending=False).head(5)

(96, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
245,146422995398181,1449004081806726,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-21,link,mostly true,NaN,9243.0,8634.0,881.0
298,146422995398181,1452319888141812,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-23,link,mostly true,NaN,7855.0,7048.0,392.0
314,146422995398181,1456117054428762,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-26,link,mostly true,NaN,6180.0,11698.0,442.0
292,146422995398181,1451988724841595,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-23,link,mostly true,NaN,5028.0,4650.0,660.0
339,146422995398181,1457326974307770,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-27,link,mostly true,yes,4000.0,6878.0,255.0


In [31]:
false_addicting_posts_mask = (addicting_posts['Rating'] == 'mostly false')
false_addicting_posts = addicting_posts[false_addicting_posts_mask]
print(false_addicting_posts.shape)
false_addicting_posts.sort_values(by='share_count', ascending=False).head(5)

(8, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
294,146422995398181,1452117881495346,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-23,link,mostly false,NaN,4708.0,13821.0,486.0
307,146422995398181,1455558234484644,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-26,link,mostly false,NaN,4587.0,7960.0,2770.0
268,146422995398181,1450712398302561,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-22,link,mostly false,NaN,3063.0,3971.0,1006.0
233,146422995398181,1448154985224969,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-20,link,mostly false,NaN,285.0,934.0,153.0
302,146422995398181,1454868161220318,left,Addicting Info,https://www.facebook.com/AddictingInfoOrg/post...,2016-09-26,link,mostly false,NaN,273.0,2423.0,301.0


In [7]:
# CNN Politics: mainstream
cnn_posts_mastk = posts['Page'].str.contains('CNN')
cnn_posts = posts[cnn_posts_mastk]
print(cnn_posts.shape)

(409, 12)


In [35]:
true_cnn_posts_mask = (cnn_posts['Rating'] == 'mostly true')
true_cnn_posts = cnn_posts[true_cnn_posts_mask]
print(true_cnn_posts.shape)
true_cnn_posts.sort_values(by='share_count', ascending=False).head(5)

(385, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
400,219367258105115,1282890125086151,mainstream,CNN Politics,https://www.facebook.com/cnnpolitics/posts/128...,2016-09-20,link,mostly true,NaN,22880.0,6899.0,1145.0
606,219367258105115,1286952664679897,mainstream,CNN Politics,https://www.facebook.com/cnnpolitics/posts/128...,2016-09-23,link,mostly true,NaN,2972.0,6980.0,5400.0
680,219367258105115,1290605144314649,mainstream,CNN Politics,https://www.facebook.com/cnnpolitics/posts/129...,2016-09-26,video,mostly true,yes,1585.0,7051.0,1467.0
697,219367258105115,1290814187627078,mainstream,CNN Politics,https://www.facebook.com/cnnpolitics/posts/129...,2016-09-27,link,mostly true,yes,1532.0,5976.0,1301.0
567,219367258105115,1286001041441726,mainstream,CNN Politics,https://www.facebook.com/cnnpolitics/posts/128...,2016-09-22,link,mostly true,NaN,1447.0,2251.0,1183.0


In [36]:
false_cnn_posts_mask = (cnn_posts['Rating'] == 'mostly false')
false_cnn_posts = cnn_posts[false_cnn_posts_mask]
print(false_cnn_posts.shape)
false_cnn_posts.sort_values(by='share_count', ascending=False).head(5)

(0, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count


In [8]:
# Eagle Rising: right
eagle_posts_mastk = posts['Page'].str.contains('Eagle')
eagle_posts = posts[eagle_posts_mastk]
print(eagle_posts.shape)

(286, 12)


In [37]:
true_eagle_posts_mask = (eagle_posts['Rating'] == 'mostly true')
true_eagle_posts = eagle_posts[true_eagle_posts_mask]
print(true_eagle_posts.shape)
true_eagle_posts.sort_values(by='share_count', ascending=False).head(5)

(121, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
798,135665053303678,551828481687331,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-20,link,mostly true,NaN,10126.0,4068.0,1641.0
824,135665053303678,552024685001044,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-20,link,mostly true,NaN,2828.0,1537.0,612.0
771,135665053303678,551518465051666,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-19,link,mostly true,NaN,2828.0,1537.0,612.0
1019,135665053303678,554510764752436,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-27,link,mostly true,NaN,2057.0,1681.0,166.0
910,135665053303678,552889718247874,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-22,link,mostly true,NaN,1944.0,1425.0,359.0


In [38]:
false_eagle_posts_mask = (eagle_posts['Rating'] == 'mostly false')
false_eagle_posts = eagle_posts[false_eagle_posts_mask]
print(false_eagle_posts.shape)
false_eagle_posts.sort_values(by='share_count', ascending=False).head(5)

(30, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
753,135665053303678,551473235056189,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-19,photo,mostly false,NaN,7577.0,2989.0,146.0
955,135665053303678,553589488177897,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-26,photo,mostly false,NaN,5870.0,2856.0,122.0
785,135665053303678,551705221699657,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-19,link,mostly false,NaN,3304.0,1996.0,220.0
897,135665053303678,552600354943477,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-22,link,mostly false,NaN,3021.0,1962.0,346.0
802,135665053303678,551828675020645,right,Eagle Rising,https://www.facebook.com/theEagleisRising/post...,2016-09-20,link,mostly false,NaN,2778.0,1063.0,165.0


In [50]:
# Freedom Daily: right 
freedom_posts_mastk = posts['Page'].str.contains('Freedom')
freedom_posts = posts[freedom_posts_mastk]
print(freedom_posts.shape)

(112, 12)


In [51]:
true_freedom_posts_mask = (freedom_posts['Rating'] == 'mostly true')
true_freedom_posts = freedom_posts[true_freedom_posts_mask]
print(true_freedom_posts.shape)
true_freedom_posts.sort_values(by='share_count', ascending=False).head(5)

(56, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
1114,440106476051475,1249978638397584,right,Freedom Daily,https://www.facebook.com/FreedomDailyNews/post...,2016-09-23,link,mostly true,NaN,40316.0,14677.0,857.0
1126,440106476051475,1252637398131708,right,Freedom Daily,https://www.facebook.com/FreedomDailyNews/post...,2016-09-26,link,mostly true,NaN,11994.0,10501.0,4376.0
1134,440106476051475,1252759804786134,right,Freedom Daily,https://www.facebook.com/FreedomDailyNews/post...,2016-09-27,link,mostly true,NaN,11754.0,10325.0,863.0
1127,440106476051475,1252638934798221,right,Freedom Daily,https://www.facebook.com/FreedomDailyNews/post...,2016-09-26,link,mostly true,NaN,8646.0,8683.0,1133.0
1129,440106476051475,1252747661454015,right,Freedom Daily,https://www.facebook.com/FreedomDailyNews/post...,2016-09-26,link,mostly true,NaN,7278.0,6616.0,385.0


In [52]:
false_freedom_posts_mask = (freedom_posts['Rating'] == 'mostly false')
false_freedom_posts = freedom_posts[false_freedom_posts_mask]
print(false_freedom_posts.shape)
false_freedom_posts.sort_values(by='share_count', ascending=False).head(5)

(26, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
1062,440106476051475,1247110935351021,right,Freedom Daily,https://www.facebook.com/FreedomDailyNews/post...,2016-09-20,link,mostly false,NaN,18054.0,17705.0,1487.0
1112,440106476051475,1249886585073456,right,Freedom Daily,https://www.facebook.com/FreedomDailyNews/post...,2016-09-23,link,mostly false,NaN,14024.0,9536.0,2247.0
1040,440106476051475,1246091905452924,right,Freedom Daily,https://www.facebook.com/FreedomDailyNews/post...,2016-09-19,link,mostly false,NaN,13305.0,9490.0,932.0
1111,440106476051475,1249777878417660,right,Freedom Daily,https://www.facebook.com/FreedomDailyNews/post...,2016-09-23,link,mostly false,NaN,6535.0,14003.0,825.0
1045,440106476051475,1246207405441374,right,Freedom Daily,https://www.facebook.com/FreedomDailyNews/post...,2016-09-19,link,mostly false,NaN,4595.0,4899.0,256.0


In [53]:
# Occupy Democrats: left
occupy_posts_mastk = posts['Page'].str.contains('Occupy')
occupy_posts = posts[occupy_posts_mastk]
print(occupy_posts.shape)

(209, 12)


In [41]:
true_occupy_posts_mask = (occupy_posts['Rating'] == 'mostly true')
true_occupy_posts = occupy_posts[true_occupy_posts_mask]
print(true_occupy_posts.shape)
true_occupy_posts.sort_values(by='share_count', ascending=False).head(5)

(102, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
1331,346937065399354,1253304291429289,left,Occupy Democrats,https://www.facebook.com/OccupyDemocrats/posts...,2016-09-27,video,mostly true,yes,322630.0,210016.0,13508.0
1285,346937065399354,1252897674803284,left,Occupy Democrats,https://www.facebook.com/OccupyDemocrats/posts...,2016-09-26,photo,mostly true,NaN,82192.0,88265.0,1398.0
1231,346937065399354,1248773611882357,left,Occupy Democrats,https://www.facebook.com/OccupyDemocrats/posts...,2016-09-22,video,mostly true,NaN,70418.0,48704.0,2549.0
1160,346937065399354,1245835812176137,left,Occupy Democrats,https://www.facebook.com/OccupyDemocrats/posts...,2016-09-19,video,mostly true,NaN,63291.0,30011.0,13510.0
1284,346937065399354,1252892884803763,left,Occupy Democrats,https://www.facebook.com/OccupyDemocrats/posts...,2016-09-26,photo,mostly true,NaN,61587.0,110656.0,1855.0


In [42]:
false_occupy_posts_mask = (occupy_posts['Rating'] == 'mostly false')
false_occupy_posts = occupy_posts[false_occupy_posts_mask]
print(false_occupy_posts.shape)
false_occupy_posts.sort_values(by='share_count', ascending=False).head(5)

(9, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
1272,346937065399354,1252678871491831,left,Occupy Democrats,https://www.facebook.com/OccupyDemocrats/posts...,2016-09-26,link,mostly false,NaN,28254.0,59251.0,2981.0
1238,346937065399354,1248526781907040,left,Occupy Democrats,https://www.facebook.com/OccupyDemocrats/posts...,2016-09-23,photo,mostly false,NaN,22075.0,34622.0,2527.0
1245,346937065399354,1249620025131049,left,Occupy Democrats,https://www.facebook.com/OccupyDemocrats/posts...,2016-09-23,link,mostly false,NaN,17635.0,22099.0,1970.0
1250,346937065399354,1249753408451044,left,Occupy Democrats,https://www.facebook.com/OccupyDemocrats/posts...,2016-09-23,link,mostly false,NaN,11931.0,39770.0,2897.0
1346,346937065399354,1254000264693025,left,Occupy Democrats,https://www.facebook.com/OccupyDemocrats/posts...,2016-09-27,link,mostly false,yes,5541.0,9110.0,638.0


In [11]:
# Politico: mainstream
politico_posts_mastk = posts['Page'].str.contains('Politico')
politico_posts = posts[politico_posts_mastk]
print(politico_posts.shape)

(536, 12)


In [43]:
true_politico_posts_mask = (politico_posts['Rating'] == 'mostly true')
true_politico_posts = politico_posts[true_politico_posts_mask]
print(true_politico_posts.shape)
true_politico_posts.sort_values(by='share_count', ascending=False).head(5)

(528, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
1725,62317591679,10153860025456680,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,2016-09-26,link,mostly true,NaN,12233.0,21425.0,1005.0
1782,62317591679,10153861466546680,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,2016-09-26,video,mostly true,yes,6857.0,28505.0,1636.0
1575,62317591679,10153848618131680,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,2016-09-22,link,mostly true,NaN,2795.0,4539.0,851.0
1808,62317591679,10153861784371680,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,2016-09-27,link,mostly true,yes,2197.0,5176.0,1521.0
1784,62317591679,10153861481676680,mainstream,Politico,https://www.facebook.com/politico/posts/101538...,2016-09-26,link,mostly true,yes,1849.0,2352.0,181.0


In [44]:
false_politico_posts_mask = (politico_posts['Rating'] == 'mostly false')
false_politico_posts = politico_posts[false_politico_posts_mask]
print(false_politico_posts.shape)
false_politico_posts.sort_values(by='share_count', ascending=False).head(5)

(0, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count


In [12]:
# Right Wing News: right
right_posts_mastk = posts['Page'].str.contains('Right')
right_posts = posts[right_posts_mastk]
print(right_posts.shape)

(268, 12)


In [46]:
true_right_posts_mask = (right_posts['Rating'] == 'mostly true')
true_right_posts = right_posts[true_right_posts_mask]
print(true_right_posts.shape)
true_right_posts.sort_values(by='share_count', ascending=False).head(5)

(142, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
2086,389658314427637,1539758062750984,right,Right Wing News,https://www.facebook.com/OfficialRightWingNews...,2016-09-26,photo,mostly true,NaN,23985.0,15112.0,1681.0
1992,389658314427637,1535358763190914,right,Right Wing News,https://www.facebook.com/OfficialRightWingNews...,2016-09-21,link,mostly true,NaN,7803.0,6281.0,723.0
2073,389658314427637,1537451599648297,right,Right Wing News,https://www.facebook.com/OfficialRightWingNews...,2016-09-23,link,mostly true,NaN,5730.0,23202.0,710.0
1897,389658314427637,1532698746790249,right,Right Wing News,https://www.facebook.com/OfficialRightWingNews...,2016-09-19,link,mostly true,NaN,5434.0,7211.0,3667.0
2127,389658314427637,1540794569314000,right,Right Wing News,https://www.facebook.com/OfficialRightWingNews...,2016-09-27,link,mostly true,NaN,3384.0,12218.0,1415.0


In [47]:
false_right_posts_mask = (right_posts['Rating'] == 'mostly false')
false_right_posts = right_posts[false_right_posts_mask]
print(false_right_posts.shape)
false_right_posts.sort_values(by='share_count', ascending=False).head(5)

(26, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
2001,389658314427637,1535215869871870,right,Right Wing News,https://www.facebook.com/OfficialRightWingNews...,2016-09-22,link,mostly false,NaN,12696.0,31112.0,1932.0
2094,389658314427637,1540141122712678,right,Right Wing News,https://www.facebook.com/OfficialRightWingNews...,2016-09-26,link,mostly false,NaN,9969.0,4700.0,746.0
2147,389658314427637,1541315515928572,right,Right Wing News,https://www.facebook.com/OfficialRightWingNews...,2016-09-27,link,mostly false,yes,6990.0,9375.0,1500.0
2104,389658314427637,1540364386023685,right,Right Wing News,https://www.facebook.com/OfficialRightWingNews...,2016-09-26,link,mostly false,NaN,5496.0,4590.0,192.0
2098,389658314427637,1540260392700751,right,Right Wing News,https://www.facebook.com/OfficialRightWingNews...,2016-09-26,link,mostly false,NaN,4846.0,7014.0,691.0


In [13]:
# The Other 98%: left
other_posts_mastk = posts['Page'].str.contains('Other')
other_posts = posts[other_posts_mastk]
print(other_posts.shape)

(122, 12)


In [48]:
true_other_posts_mask = (other_posts['Rating'] == 'mostly true')
true_other_posts = other_posts[true_other_posts_mask]
print(true_other_posts.shape)
true_other_posts.sort_values(by='share_count', ascending=False).head(5)

(67, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
2269,114517875225866,1472059186138388,left,The Other 98%,https://www.facebook.com/TheOther98/posts/1472...,2016-09-27,video,mostly true,yes,211158.0,73348.0,13530.0
2270,114517875225866,1472225646121742,left,The Other 98%,https://www.facebook.com/TheOther98/posts/1472...,2016-09-27,photo,mostly true,yes,71217.0,149263.0,3064.0
2178,114517875225866,1463586530318987,left,The Other 98%,https://www.facebook.com/TheOther98/posts/1463...,2016-09-20,photo,mostly true,NaN,50881.0,47190.0,1404.0
2168,114517875225866,1462710973739876,left,The Other 98%,https://www.facebook.com/TheOther98/posts/1462...,2016-09-19,link,mostly true,NaN,33767.0,27185.0,3129.0
2162,114517875225866,1462507497093557,left,The Other 98%,https://www.facebook.com/TheOther98/posts/1462...,2016-09-19,link,mostly true,NaN,26726.0,20354.0,1471.0


In [49]:
false_other_posts_mask = (other_posts['Rating'] == 'mostly false')
false_other_posts = other_posts[false_other_posts_mask]
print(false_other_posts.shape)
false_other_posts.sort_values(by='share_count', ascending=False).head(5)

(5, 12)


,account_id,post_id,Category,Page,Post URL,Date Published,Post Type,Rating,Debate,share_count,reaction_count,comment_count
2248,114517875225866,1470814329596207,left,The Other 98%,https://www.facebook.com/TheOther98/posts/1470...,2016-09-26,link,mostly false,NaN,32181.0,48387.0,2551.0
2213,114517875225866,1465108913500082,left,The Other 98%,https://www.facebook.com/TheOther98/posts/1465...,2016-09-21,photo,mostly false,NaN,21673.0,19682.0,329.0
2190,114517875225866,1463890730288567,left,The Other 98%,https://www.facebook.com/TheOther98/posts/1463...,2016-09-20,photo,mostly false,NaN,11571.0,8568.0,930.0
2189,114517875225866,1463842706960036,left,The Other 98%,https://www.facebook.com/TheOther98/posts/1463...,2016-09-20,photo,mostly false,NaN,3261.0,9590.0,445.0
2174,114517875225866,1462891757055131,left,The Other 98%,https://www.facebook.com/TheOther98/posts/1462...,2016-09-20,photo,mostly false,NaN,3.0,36556.0,1002.0
